In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import glob
import xml.etree.ElementTree as ET
import fnmatch
import os,sys
import random
from PIL import Image

In [32]:
def parse_rec(filename):
    """ Parse a stateDetection xml file """
    tree = ET.parse(filename)
    objects = []
    for obj in tree.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        obj_struct['difficult'] = int(obj.find('difficult').text)
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text),
                              int(bbox.find('ymin').text),
                              int(bbox.find('xmax').text),
                              int(bbox.find('ymax').text)]
        objects.append(obj_struct)

    return objects

In [33]:
# path to image set file name train/val.txt contains image name in each line
imagesetfile = '/home/ashu/Study Material/Uni Bonn/4th Semester/Thesis/DataSet/coco-master/ImageSets/Main/train.txt'
#imagesetfile = '/home/ashu/Study Material/Uni Bonn/4th Semester/Thesis/DataSet/coco-master/ImageSets/Main/sample_train.txt'
with open(imagesetfile, 'r') as f:
        lines = f.readlines()
imagenames = [x.strip() for x in lines]

annopath = os.path.join(
            'Annotations',
            '{:s}.xml')
#load all annotations 
recs = {}
for i, imagename in enumerate(imagenames):
    recs[imagename] = parse_rec(annopath.format(imagename))
    if i % 1000 == 0:
        print 'Reading annotation for {:d}/{:d}'.format(
                i + 1, len(imagenames))
print len(recs)

Reading annotation for 1/9662
Reading annotation for 1001/9662
Reading annotation for 2001/9662
Reading annotation for 3001/9662
Reading annotation for 4001/9662
Reading annotation for 5001/9662
Reading annotation for 6001/9662
Reading annotation for 7001/9662
Reading annotation for 8001/9662
Reading annotation for 9001/9662
9662


In [58]:
class_name = 'umbrella'

#Extract gt objects of this class only
class_recs = {}
npos = 0
for imagename in imagenames:
        R = [obj for obj in recs[imagename] if obj['name'].split('_')[0] == class_name]
        
        #extracting state name
        state = [x['name'].split('_')[1] for x in R]
        bbox = np.array([x['bbox'] for x in R])
        det = [False] * len(R)
        
        # generate state labels
        label = np.array([0 if 'open' in st else 1 for st in state])  
        #print state
        #print label
        difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
        npos = npos + sum(~difficult)
        class_recs[imagename] = {'bbox': bbox, # all gt bbox of image
                                 'difficult': difficult,  # every object is marked as not difficult
                                 'det': det,    # mark all bbox as not detected
                                 'label': label} # state labels for classifier
        

In [59]:
# check number of bbox in class, must be equal to number of objects in val set of that class
count = 0
for imagename in imagenames:
    R = class_recs[imagename]
    BBGT = R['bbox'].astype(float)
    if BBGT.size > 0: # i.e. image contains the box
        count += len(BBGT)
print 'Total objects in ', class_name, ': ', count

Total objects in  umbrella :  7814


In [60]:
# Root folder for Detections
det_root = 'det_train'
images_root = '/home/ashu/Study Material/Uni Bonn/4th Semester/Thesis/DataSet/coco-master/JPEGImages/'

# creating file name for open and closed state
if class_name == 'scissors':
    class_name = 'scissor'
    
det_file = det_root + '/' + det_root + '_' + class_name  + '.txt'
    
print det_file

det_train/det_train_umbrella.txt


In [61]:
# load detections
#open_file = os.path.join('det_val','det_val_laptop_closedLaptop.txt')
#close_file = os.path.join('det_val','det_val_laptop_openLaptop.txt')

with open(det_file, 'r') as f:
    lines = np.array(f.readlines())

#with open(close_file, 'r') as f:
#    close_det = np.array(f.readlines())

remove_new_line = np.vectorize(lambda x: x.strip())

lines = remove_new_line(lines)
#Assign labels to Open and close detection open :0, close: 1
#train_det = np.core.defchararray.add(remove_new_line(train_det), " 0")
#close_det = np.core.defchararray.add(remove_new_line(close_det), " 1")

#open_det = list(open_det)
#close_det = list(close_det)

# merge list
#lines = open_det 

#print len(open_det),  len(lines)
#print open_det[:3], close_det[-3:]
print lines[:3], lines[-3:]

['COCO_train2014_000000296735 0.845 198.5 278.9 239.2 302.3'
 'COCO_train2014_000000296735 0.382 190.7 268.7 286.0 297.4'
 'COCO_train2014_000000296735 0.146 431.8 259.7 452.4 323.1'] ['COCO_train2014_000000042169 0.160 80.6 165.3 254.2 255.9'
 'COCO_train2014_000000042169 0.125 40.3 383.9 143.2 444.5'
 'COCO_train2014_000000042169 0.115 127.4 238.2 269.1 353.0']


In [62]:
# create folders for storing images and labels
image_dir = det_root + '/images/' + class_name + '/'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
        
labels_dir = det_root + '/labels/'
if not os.path.exists(labels_dir):
    os.makedirs(labels_dir)
    
label_fileName = labels_dir + class_name + '_' + det_root[4:] + '.txt'
print label_fileName

det_train/labels/umbrella_train.txt


In [63]:
ovthresh = 0.5
splitlines = [x.strip().split(' ') for x in lines]
image_ids = [x[0] for x in splitlines]
confidence = np.array([float(x[1]) for x in splitlines])
BB = np.array([[float(z) for z in x[2:6]] for x in splitlines])



# sort by confidence
sorted_ind = np.argsort(-confidence)
sorted_scores = np.sort(-confidence)
BB = BB[sorted_ind, :]
image_ids = [image_ids[x] for x in sorted_ind]

# go down dets and mark TPs and FPs
nd = len(image_ids)
tp = np.zeros(nd)
fp = np.zeros(nd)

print nd
# looping through detections
labels_list = []
for d in range(nd):
    R = class_recs[image_ids[d]]
    bb = BB[d, :].astype(float) # Detected bbox
    ovmax = -np.inf
    BBGT = R['bbox'].astype(float) # GT bboxes for same image

    if BBGT.size > 0:
        # compute overlaps
        # intersection
        ixmin = np.maximum(BBGT[:, 0], bb[0])
        iymin = np.maximum(BBGT[:, 1], bb[1])
        ixmax = np.minimum(BBGT[:, 2], bb[2])
        iymax = np.minimum(BBGT[:, 3], bb[3])
        iw = np.maximum(ixmax - ixmin + 1., 0.)
        ih = np.maximum(iymax - iymin + 1., 0.)
        inters = iw * ih

        # union
        uni = ((bb[2] - bb[0] + 1.) * (bb[3] - bb[1] + 1.) +
               (BBGT[:, 2] - BBGT[:, 0] + 1.) *
                (BBGT[:, 3] - BBGT[:, 1] + 1.) - inters)

        overlaps = inters / uni
        ovmax = np.max(overlaps)  # taking the gt bbox that has max overlap
        jmax = np.argmax(overlaps)  # taking the index of max overlap bbox
    
    # check if IoU crosses threshold
    if ovmax > ovthresh:
        tp[d] = 1.
        R['det'][jmax] = 1 # marking that bbox as detected
                    
        # crop image, assign new name and save in folder
        img = Image.open(images_root + image_ids[d] + '.jpg')
        new_img = img.crop((bb[0], bb[1], bb[2], bb[3]))
        new_img_name = image_ids[d] + '_' + str(d) + '.jpg'
        new_img.save(image_dir + new_img_name)
                
        # generate labeland add to list
        lbl = str(new_img_name + ' ' + str(R['label'][jmax]) + '\n')
        labels_list.append(lbl)

    else:
        fp[d] = 1.

print np.sum(tp)
#write labels into file
with open(label_fileName, 'w+') as f:
    for val in labels_list:
        f.write(val)

36965
5491.0
